In [1]:
#Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Initialize the base URL and a DataFrame to store all data
base_url = "https://gpessays.sg/gp-model-essays/"
all_data = []

# Function to scrape article information
def scrape_article(link):
    response2 = requests.get(link)
    soup2 = BeautifulSoup(response2.text, 'html.parser')
    
    title = soup2.find("h1", class_="entry-title").get_text(strip=True)
    
    # Capture meta information
    meta_info = soup2.find_all("div", style="margin-top: 15px;")[0]
    lines = meta_info.decode_contents().split('<br/>')
    
    meta_dict = {}
    for line in lines:
        line_soup = BeautifulSoup(line, 'html.parser')
        key_tag = line_soup.find('strong')
        if key_tag and key_tag.next_sibling:
            key = key_tag.get_text(strip=True).replace(':', '').strip().lower().replace(' ', '_')
            value = key_tag.next_sibling.strip()
            meta_dict[key] = value
    
    # Extract essay content
    entry_div = soup2.find('div', class_='entry-content')
    text = entry_div.text.replace('\n', ' ').strip()
    meta_dict['essay'] = text
    
    return meta_dict

# Function to scrape each page
def scrape_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    articles = soup.find_all("div", class_="inside-article")
    
    # Loop through articles and scrape data
    for article in articles:
        title_tag = article.find("h2", class_="entry-title").find("a")
        title = title_tag.text.strip()
        link = title_tag['href']
        
        # Scrape article details
        article_data = scrape_article(link)
        article_data['title'] = title
        all_data.append(article_data)

# Function to handle pagination and scrape all pages
def scrape_all_pages(base_url):
    page_num = 1
    while True:
        # Construct the URL for the current page
        url = f"{base_url}page/{page_num}/" if page_num > 1 else base_url
        
        print(f"Scraping page {page_num}...")
        scrape_page(url)
        
        # Check if there's a next page
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        next_page = soup.find("a", class_="next")
        
        if next_page:
            page_num += 1
        else:
            break

# Start scraping from the first page
scrape_all_pages(base_url)

# Convert collected data to a DataFrame
df = pd.DataFrame(all_data)

# Show the DataFrame or save it to a CSV
df.head() # Display the first few rows

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...


,author,school,year_written,grade,essay,title
0,Skyler,Hwa Chong Institution,2021,A,Hong Kong no doubt boasts some of the wealthie...,‘Prosperity is all that matters; everything el...
1,Skyler,Hwa Chong Institution,2021,A,"Freedom of speech is, to the Western world, a ...",Do you agree that freedom of speech should nev...
2,Skyler,Hwa Chong Institution,2021,A,"Since time immemorial, humanity has been plagu...","‘When children grow up exposed to violence, th..."
3,Junwei,Hwa Chong Institution,2020,36,The 2020 U.S. Presidential Election seemed to ...,A government that fails to win the trust of th...
4,Junwei,Hwa Chong Institution,2020,35/50,"In the aftermath of the Second World War, the ...",To what extent is the use of violence in today...


In [3]:
df.to_csv("scraped_essays.csv", index=False) 